In [6]:
import os
import requests
import pandas as pd
import time

In [2]:
API_KEY = os.getenv("YELP_API_KEY")
if API_KEY is None:
    raise ValueError("YELP_API_KEY environment variable not set.")

HEADERS = {"Authorization": f"Bearer {API_KEY}"}

In [3]:

# --- PAGINATED SEARCH FUNCTION ---
def search_yelp_paginated(term="thrift_stores", location="Provo, UT",
                          limit=50, max_results=1000):
    """
    Collects up to max_results Yelp businesses using pagination.
    - term: Yelp alias (e.g., 'thrift_stores')
    - limit: max 50 per Yelp API request
    - max_results: Yelp Search caps near 1000 items max
    """
    url = "https://api.yelp.com/v3/businesses/search"

    all_businesses = []
    offset = 0

    while True:
        params = {
            "term": term,
            "location": location,
            "limit": limit,
            "offset": offset,
        }

        response = requests.get(url, headers=HEADERS, params=params)
        response.raise_for_status()

        data = response.json()
        businesses = data.get("businesses", [])

        # Stop if no more results
        if not businesses:
            break

        all_businesses.extend(businesses)

        # Increase offset
        offset += limit

        # Yelp won't return beyond ~1000 results
        if offset >= max_results:
            break

        # Avoid hitting rate limits
        time.sleep(0.25)

    return all_businesses

In [4]:
# --- CITIES TO SEARCH ---
cities = [
    # Utah County
    "Provo, UT",
    "Orem, UT",
    "Lehi, UT",
    "American Fork, UT",
    "Pleasant Grove, UT",
    "Spanish Fork, UT",
    "Springville, UT",
    "Lindon, UT",
    "Highland, UT",

    # Salt Lake County
    "Salt Lake City, UT",
    "West Valley City, UT",
    "Sandy, UT",
    "Draper, UT",
    "West Jordan, UT",
    "South Jordan, UT",
    "Midvale, UT",
    "Murray, UT",
]


In [7]:
# --- COLLECT DATA ---
all_data = []
for city in cities:
    print(f"Collecting thrift stores from: {city} ...")
    results = search_yelp_paginated(
        term="thrift_stores",
        location=city,
        limit=50,
        max_results=1000
    )

    for b in results:
        all_data.append({
            "name": b.get("name"),
            "rating": b.get("rating"),
            "review_count": b.get("review_count"),
            "price": b.get("price"),
            "categories": ", ".join([c["title"] for c in b.get("categories", [])]),
            "latitude": b.get("coordinates", {}).get("latitude"),
            "longitude": b.get("coordinates", {}).get("longitude"),
            "address": " ".join(b.get("location", {}).get("display_address", [])),
            "city_queried": city
        })

In [8]:

# --- SAVE RESULTS ---
df = pd.DataFrame(all_data).drop_duplicates(subset=["name", "address"])
df.to_csv("utah_thrift_stores.csv", index=False)

df.head()

,name,rating,review_count,price,categories,latitude,longitude,address,city_queried
0,Thrifthood,4.3,12,None,Thrift Stores,40.231670,-111.661290,"160 S 100 W St Ste 1 Provo, UT 84601","Provo, UT"
1,Making Space Thrift,4.3,3,None,"Thrift Stores, Fabric Stores",40.240120,-111.662942,"475 N Freedom Blvd Provo, UT 84601","Provo, UT"
2,Get Thrifty,3.4,8,None,Thrift Stores,40.256653,-111.670738,"1700 N State Street Ste 20 Provo, UT 84604","Provo, UT"
3,Curveture,5.0,2,None,"Plus Size Fashion, Thrift Stores, Women's Clot...",40.252912,-111.661560,"1394 N Freedom Blvd Provo, UT 84604","Provo, UT"
4,Preloved Provo,4.9,7,None,"Thrift Stores, Men's Clothing, Women's Clothing",40.250892,-111.662831,"230 Cougar Blvd Provo, UT 84604","Provo, UT"
